In [ ]:
!source .venv/bin/activate
!pip3 install kagglehub




In [2]:
!pip3 install cudf-cu11 --extra-index-url=https://pypi.nvidia.com
!pip3 install cudf-cu12 --extra-index-url=https://pypi.nvidia.com
!pip3 install -U polars[gpu] --extra-index-url=https://pypi.nvidia.com

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com
  Using cached https://pypi.nvidia.com/cudf-cu11/cudf_cu11-24.10.1-cp310-cp310-manylinux_2_24_x86_64.manylinux_2_28_x86_64.whl (24.3 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.9/19.9 MB 9.2 MB/s eta 0:00:00:00:010:01
  Using cached https://pypi.nvidia.com/pylibcudf-cu11/pylibcudf_cu11-24.10.1-cp310-cp310-manylinux_2_24_x86_64.manylinux_2_28_x86_64.whl (20.0 MB)
     ━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━━━ 219.5/502.8 MB 4.8 MB/s eta 0:01:00
ERROR: THESE PACKAGES DO NOT MATCH THE HASHES FROM THE REQUIREMENTS FILE. If you have updated the package versions, please update the hashes. Otherwise, examine the package contents carefully; someone may have tampered with them.
    libcudf-cu11==24.10.* from https://pypi.nvidia.com/libcudf-cu11/libcudf_cu11-24.10.1-py3-none-manylinux_2_28_x86_64.whl#sha256=d6fc0d34e25f162fcd11db40152425e8e5f17

In [ ]:
import kagglehub
import os
import shutil

# Download latest version
path = kagglehub.dataset_download("asaniczka/tmdb-movies-dataset-2023-930k-movies")

print("Path to dataset files:", path)


ruta = path  # Cambia esta ruta por el directorio que quieras listar

# Listar archivos y carpetas en la ruta
archivos = os.listdir(ruta)

# Filtrar solo archivos
solo_archivos = [f for f in archivos if os.path.isfile(os.path.join(ruta, f))]

print("Archivos en el directorio:")
for archivo in solo_archivos:
    print(archivo)
    # Copiar archivo
    try:
        shutil.copy(path+'/'+ archivo, 'dataSets'+'/'+archivo)
        print("Archivo copiado exitosamente.")
    except FileNotFoundError:
        print("El archivo de origen no fue encontrado.")
    except PermissionError:
        print("No tienes permisos para acceder a la ubicación de destino.")
    except Exception as e:
        print(f"Ocurrió un error al copiar el archivo: {e}")

Path to dataset files: /home/fernando/.cache/kagglehub/datasets/asaniczka/tmdb-movies-dataset-2023-930k-movies/versions/382
Archivos en el directorio:
TMDB_movie_dataset_v11.csv
Archivo copiado exitosamente.


In [10]:
import polars as pl

df = pl.scan_csv('dataSets/'+archivo)

print(df)

naive plan: (run LazyFrame.explain(optimized=True) to see the optimized plan)

Csv SCAN [dataSets/TMDB_movie_dataset_v11.csv]
PROJECT */24 COLUMNS


In [12]:
import pandas as pd

df_pandas = pd.read_csv('dataSets/'+archivo)
print(df_pandas)

              id  ...                                           keywords
0          27205  ...  rescue, mission, dream, airplane, paris, franc...
1         157336  ...  rescue, future, spacecraft, race against time,...
2            155  ...  joker, sadism, chaos, secret identity, crime f...
3          19995  ...  future, society, culture clash, space travel, ...
4          24428  ...  new york city, superhero, shield, based on com...
...          ...  ...                                                ...
1127772   728735  ...                                                NaN
1127773   728737  ...                                                NaN
1127774   728738  ...                                                NaN
1127775   728739  ...                                                NaN
1127776  1382677  ...                                                NaN

[1127777 rows x 24 columns]


In [15]:
gpu_engine = pl.GPUEngine(
    device=0, # This is the default
    raise_on_fail=True, # Fail loudly if we can't run on the GPU.
)

In [16]:
df.select(pl.col("keywords").sum()).collect(engine=gpu_engine)

AttributeError: module 'cuda.ccudart' has no attribute '__pyx_capi__'